# TP-2 Organización de Datos

# Oh La La... Data!


* **Aymeryc COUSAERT** ------  *Padrón 105464 - Alumno regular de intercambio.*
* **Mariana VINYOLAS**  ---------  *Alumna oyente.*

<!-- ![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
<a id='section2'></a>

<div style="border-bottom:3px solid #000;"> 
<div align="left"><h2>Step 2: Import de librerias </h2></div>
<div align="right">(return to <a href='#top1'><b>Top</b></a>)</div>
</div> -->



In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns


import re
import nltk.corpus
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import catboost as catb
import lightgbm as lgbm

import scikitplot as skplt
from scikitplot.metrics import plot_roc
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

C:\anaconda\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


<!-- ![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
<a id='section3'></a>

<div style="border-bottom:3px solid #000;"> 
<div align="left"><h2>Step 3: Feature engineering</h2></div>
<div align="right">(return to <a href='#top1'><b>Top</b></a>)</div>
</div> -->

In [5]:

df = train
df['text'] = df['text'].str.lower()   # pasamos a lowercase
df = df.drop(['id', 'location'], axis=1)

In [6]:
df.sample(8)

,keyword,text,target
5468,quarantine,reddit updates content policy promises to quar...,0
6997,twister,crazy mom threw teen daughter a nude twister s...,1
3089,electrocute,kayla is about to electrocute herself.,0
3396,evacuation,gas leak forces evacuation in east saint john ...,1
1113,bombed,@christopherszen @hunterlove1995 @tblack yeah ...,0
1600,collapse,warne ponting shocked by australian collapse -...,0
6779,tragedy,there is no greater tragedy than becoming comf...,0
7459,wounds,court back in session. testimony continues wit...,0


#### Generamos nuevas features sobre el texto

In [7]:
# generamos una columna que indica la cantidad de links a  enlaces externos
df['link'] = df['text'].apply(lambda x: x.count('http'))

# generamos una columna que indica la cantidad de referencias a otras cuentas de twitter
df['contact'] = df['text'].apply(lambda x: x.count('@'))

# generamos una columna que indica la cantidad de hashtags
df['hashtag'] = df['text'].apply(lambda x: x.count('#'))

# generamos una columna que indica la cantidad de digitos
df['numerics'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

# calculamos la longitud del tweet andes de limpiar
df['length'] = df['text'].str.len()

In [8]:
# calculamos la cantidad de palabras antes de limpiar
def count_words(text):
    '''
    Funcion que toma un texto y devuelve la cantidad de palabras
    '''
    word_counts = len(text.split(' '))
    return word_counts

df['words'] = df['text'].apply(count_words)

In [9]:
# limpiamos el texto eliminando urls, cuentas, hashtags y numeros

def clean_str(string):
    string = re.sub(r'https?\://\S+', '', string)
    string = re.sub(r'http?\://\S+', '', string)
    string = re.sub(r'@\w*\s', '', string)
    string = re.sub(r'#\w*\s', '', string)
    string = re.sub(r'\d', '', string)
    return string

df['text_clean'] = df['text'].apply(lambda x: clean_str(str(x)))

In [10]:
df.sample(5)

,keyword,text,target,link,contact,hashtag,numerics,length,words,text_clean
7152,volcano,this la startup is so hot that their flowers c...,0,1,1,0,0,113,17,this la startup is so hot that their flowers c...
7386,windstorm,new roof and hardy up..windstorm inspection to...,0,1,0,0,0,75,8,new roof and hardy up..windstorm inspection to...
6894,traumatised,@disneyirh so traumatised im ???? http://t.co/...,1,1,1,0,0,56,6,so traumatised im ????
1682,collide,@mattcohen4fake gamma ray january worlds colli...,0,0,1,0,0,129,22,gamma ray january worlds collide she waits be ...
3745,fire,my asshole is on fire https://t.co/y3fo0ghg8t,0,1,0,0,0,46,7,my asshole is on fire


<!-- ![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
<a id='section4'></a>

<div style="border-bottom:3px solid #000;"> 
<div align="left"><h2>Step 4: NLP</h2></div>
<div align="right">(return to <a href='#top1'><b>Top</b></a>)</div>
</div> -->

In [11]:
# eliminamos stopwords

stop = stopwords.words('english')

df['text_clean'] = df['text_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [12]:
# eliminamos signos de puntuacion y caracteres especiales
df['text_clean'] = df['text_clean'].str.replace('[^\w\s]','')

In [13]:
df['word_tokenize'] = df['text_clean'].apply(lambda x: word_tokenize(x))

In [14]:
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

In [15]:
# unificamos palabras que poseen la misma raiz aplicando la funcion word_lematizer
df['word_lemmatizer'] = df['word_tokenize'].apply(lambda x: word_lemmatizer(x))

In [16]:
# unificamos la lista de tokens para poder analizar el texto limpio
df['text_clean'] = df['word_lemmatizer'].str.join(' ')

In [17]:
# calculamos nuevamente la longuitud, pero ahora del texto limpio                       
df['length-clean'] = df['text_clean'].str.len()

In [18]:
df['words_clean'] = df['text_clean'].apply(count_words)

In [19]:
pd.set_option('display.max_colwidth', 150)  

In [20]:
df[['text', 'text_clean']].sample(10)

,text,text_clean
4343,another mac vuln!\n\nhttps://t.co/oxxrnab8un,another mac vuln
4976,#3: titan warriorcord 100 feet - authentic military 550 paracord - mil-c-5040-h type iii 7 strand 5/16' di... http://t.co/eejrmktj0r,titan warriorcord foot authentic military paracord milch type iii strand di
6478,@untameddirewolf 'i... wow. alright.' sansa shook her head and blinked rapidly as the new information sunk in. 'i really don't know what--,i wow alright sansa shook head blinked rapidly new information sunk in i really know what
6593,remembrance http://t.co/ii4ewe1qir #hiroshima http://t.co/h3vusqzyqo,remembrance
1053,womens handbags cross body geometric pattern satchel totes shoulder bags white http://t.co/l1gfxgozvx http://t.co/tkjybjjskl,woman handbag cross body geometric pattern satchel tote shoulder bag white
1762,west valley i405 n / us101 s i405 n con **trfc collision-unkn inj** http://t.co/js9ehp88wq,west valley n u n con trfc collisionunkn inj
6633,update on pulwama encounter that began earlier today: 2nd terrorist killed by security forces. security forces... http://t.co/m5rjekvddp,update pulwama encounter began earlier today nd terrorist killed security force security force
7049,4yygb mhtw4fnet\n\nthousands evacuated as taiwan prepares for strongest typhoon of 2015 - abc online,yygb mhtwfnet thousand evacuated taiwan prepares strongest typhoon abc online
7612,the latest: more homes razed by northern california wildfire - abc news http://t.co/ymy4rskq3d,latest home razed northern california wildfire abc news
477,heart disease prevention: what about secondhand smoke? http://t.co/ydgmgbryl2,heart disease prevention secondhand smoke


In [21]:
df['keyword'] = df.keyword.str.replace('%20', '_')
df['keyword'] = df.keyword.str.lower()

In [22]:
df.sample(3)

,keyword,text,target,link,contact,hashtag,numerics,length,words,text_clean,word_tokenize,word_lemmatizer,length-clean,words_clean
5150,nuclear_reactor,a 17 year boy scout created a mini nuclear reactor in his home,0,0,0,0,1,62,13,year boy scout created mini nuclear reactor home,"[year, boy, scout, created, mini, nuclear, reactor, home]","[year, boy, scout, created, mini, nuclear, reactor, home]",48,8
1648,collapsed,petition | heartless owner that whipped horse until it collapsed is told he can keep his animal! act now! http://t.co/njrjxqbjr4,0,1,0,0,0,128,20,petition heartless owner whipped horse collapsed told keep animal act now,"[petition, heartless, owner, whipped, horse, collapsed, told, keep, animal, act, now]","[petition, heartless, owner, whipped, horse, collapsed, told, keep, animal, act, now]",73,11
1643,collapsed,@indiepopmom i cant breathe my lungs collapsed,0,0,1,0,0,46,7,cant breathe lung collapsed,"[cant, breathe, lungs, collapsed]","[cant, breathe, lung, collapsed]",27,4


## TF- IDF

In [23]:
# aplicamos TF-IDF seteando un maximo de 1500 palabras
tfidf = TfidfVectorizer(max_features=1500, lowercase=True, analyzer='word', stop_words= 'english',ngram_range=(1,1))


train_vect = tfidf.fit_transform(df['text_clean'])                                             

# lo pasamos a dataframe
df_tf_idf = pd.DataFrame(data = train_vect.todense(), columns = tfidf.get_feature_names())

In [24]:
df_tf_idf.shape

(7613, 1500)

In [25]:
df_tf_idf['target'].value_counts()

0.000000    7602
0.750117       3
0.317217       1
0.769954       1
0.758087       1
0.322396       1
0.601328       1
0.513594       1
0.493300       1
0.437970       1
Name: target, dtype: int64

In [26]:
df_tf_idf.head()

,aba,abandoned,abc,ablaze,able,absolutely,access,accident,according,account,...,york,young,youth,youtube,yr,zone,û_,ûïwhen,ûò,ûó
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
df_tf_idf['target'].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: target, dtype: float64

In [28]:
# como existe la columna target en el analisis tf-idf, la modifico para hacer el concat con 'target' y que no se duplique
df_tf_idf['targ'] = df_tf_idf['target']
df_tf_idf = df_tf_idf.drop(['target'], axis=1)

Para no perder tantos registros, vamos a dejar de lado por ahora la columna ``keyword`` 

In [29]:
df['target'].value_counts()  # chequeamos si los labels estan balanceados

0    4342
1    3271
Name: target, dtype: int64

In [30]:
# eliminamos col que contienen texto y conservamos unicamente las numericas
df_num = df.drop([ 'keyword', 'text', 'text_clean', 'word_tokenize', 'word_lemmatizer'], axis=1)

In [31]:
df_train = pd.concat([df_num, df_tf_idf], axis=1)  # revisar porque aparecen dos columnas llamadas target

In [32]:
df_train.shape

(7613, 1509)

## Train Test Split

In [33]:
# separamos el target del resto de los features

y = df_train.target    
X = df_train.drop('target', axis=1)

In [34]:
X.shape, y.shape

((7613, 1508), (7613,))

In [35]:
# hacemos division entre train y test para cross validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)      

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5329, 1508), (2284, 1508), (5329,), (2284,))

In [36]:
# estandarizamos las features
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test) 

In [37]:
# definimos una Crossval
cv = StratifiedKFold(n_splits=5, random_state=27, shuffle=True)

## LGBM

In [40]:
params = {
    'learning_rate': [0.1,0.2, 0.3, 0.4],
    'num_leaves':[10, 15, 20],
    'max_depth':[ 18, 20, 25, 30], 
    'n_estimators': [100, 130, 150]  
    }

model_lgbm=lgbm.LGBMClassifier(n_jobs=-1, random_state=27,  boosting_type='gbdt')

In [41]:
%%time
lgbm_cv = RandomizedSearchCV(model_lgbm, params, cv=cv, n_iter=25, verbose=False, scoring='accuracy', random_state=27)
                                                                                          # 'neg_log_loss'
lgbm_cls = lgbm_cv.fit(X_train, y_train)

Wall time: 24min 8s


In [42]:
lgbm_cls.best_params_

{'num_leaves': 10, 'n_estimators': 130, 'max_depth': 20, 'learning_rate': 0.3}

In [43]:
pred_search = lgbm_cv.predict(X_test)
print(accuracy_score(y_test, pred_search))

0.7692644483362522


In [44]:
0.7661996497373029

0.7661996497373029

In [45]:
0.7653239929947461

0.7653239929947461